In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import transforms
from torchsummary import summary
import tltorch

In [2]:
path = "/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best"
model = torch.load(path)

In [3]:
def decompose_conv(m):
    return tltorch.FactorizedConv.from_conv(
        m, 
        rank=0.5, 
        decompose_weights=True, 
        factorization='tucker',
    )


def nested_children(m: torch.nn.Module, layers=['conv1']):
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            if type(child) == torch.nn.modules.conv.Conv2d and name in layers:
                m._modules[name] = decompose_conv(child)
            try:
                output[name] = nested_children(child, layers=layers)
            except TypeError:
                output[name] = nested_children(child, layers=layers)
    return output


def low_rank_tucker(model, **kwargs):
    out = nested_children(model, **kwargs)

model = torch.load(path)
low_rank_tucker(model, layers=["conv2"])
print(model)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (shortcut): Sequential()
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): B

In [6]:
def low_rank(m):
    for name, mod in m.named_children():
        if type(mod) == torch.nn.modules.conv.Conv2d:
            mod = tltorch.FactorizedConv.from_conv(
                mod, 
                rank=0.5, 
                decompose_weights=True, 
                factorization='tucker',
            )
            m._modules[name] = mod
model = torch.load(path)
model.apply(low_rank)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


ResNet(
  (conv1): FactorizedConv(
    in_channels=1, out_channels=64, kernel_size=(3, 3), rank=(4, 1, 1, 1), order=2, padding=[1, 1], bias=False
    (weight): TuckerTensor(shape=(64, 1, 3, 3), rank=(4, 1, 1, 1))
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )


In [ ]:
def decompose_conv(m):
    return tltorch.FactorizedConv.from_conv(
        m, 
        rank=0.5, 
        decompose_weights=True, 
        factorization='tucker',
    )

def d_nested_children(m: torch.nn.Module):
    if type(m) == torch.nn.modules.conv.Conv2d:
        m = decompose_conv(m)
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            try:
                output[name] = d_nested_children(child)
            except TypeError:
                output[name] = d_nested_children(child)
    return output

model = torch.load(path)
d_nested_children(model)

In [ ]:
def get_children(m: torch.nn.Module):
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            try:
                output[name] = get_children(child)
            except TypeError:
                output[name] = get_children(child)
    return output

In [35]:
def ch_dict(mod):
    return {name: mod for name, mod in mod.named_children()}

def gen_rec_child(ch):
    def rec_child(m, ch={}):
        for name, mod in m.named_children():
            check = len(ch_dict(mod)) 
            check is None:
                {}
            {}= blah
        
    return rec_child(ch)


In [66]:
model.modules()
model.children()
model.named_children()
model.named_modules()



<generator object Module.named_children at 0x7f999f05d820>

In [84]:
i = model.named_children()

while True:
    name, layer = next(i)
    print(name)
    # print(layer)
    print("-"*10)
    
    if "layer" in name:
        it = layer.named_children()
        for r in range(2):
            name, layer = next(it)
            print(name)
            print(layer)




conv1
----------
bn1
----------
layer1
----------
0
PreActBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (shortcut): Sequential()
)
1
PreActBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (shortcut): Sequential()
)
layer2
----------
0
PreActBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 128, k

StopIteration: 

In [47]:
def get_children(m):
    return {k:v for k,v in m.named_children()}

In [71]:
dic1 = get_children(model)


for k,v in model.named_children():
    dic2 = get_children(v)
    if len(dic2) == 0:
        continue
    else:
        for k2,v2 in v.named_children():
            dic3 = get_children(v2)
            if len(dic3) == 0:
                continue
            dic2[k2] = dic3
        dic1[k] = dic2
    


In [72]:
dic1

{'conv1': Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 'layer1': {'0': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()},
  '1': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()}},
 'layer2': {'0

In [75]:
def re(m):
    dic = get_children(m)
    flat = []
    for k,v in model.named_children():
        dic[k] = v
        ch = get_children(v)
        if len(ch):
            

    return dic



In [85]:
def get_children(model: torch.nn.Module):
    # get children form model!
    children = dict(model.named_children())
    flatt_children = {}
    if children == {}:
        # if model has no children; model is last child! :O
        return model
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            try:
                flatt_children[name] = get_children(child)
            except TypeError:
                flatt_children[name] = get_children(child)
    return flatt_children

In [86]:
get_children(model)

{'conv1': Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 'layer1': {'0': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()},
  '1': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()}},
 'layer2': {'0

In [82]:
dict(model.named_children())

{'conv1': Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 'layer1': Sequential(
   (0): PreActBlock(
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (shortcut): Sequential()
   )
   (1): PreActBlock(
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),

In [92]:
def nested_children(m: torch.nn.Module):
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            try:
                output[name] = nested_children(child)
            except TypeError:
                output[name] = nested_children(child)
    return output

In [95]:
nested_children(model)

{'conv1': Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 'layer1': {'0': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()},
  '1': {'bn1': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv1': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'bn2': BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   'conv2': Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   'shortcut': Sequential()}},
 'layer2': {'0

In [107]:
model = torch.load("/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best")
nested = nested_children(model)
conv = nested['layer1']['0']['conv1']

In [145]:
def decompose(m):
    if type(m) == torch.nn.modules.conv.Conv2d:
        m = tltorch.FactorizedConv.from_conv(
            m, 
            rank=0.5, 
            decompose_weights=True, 
            factorization='tucker',
        )
    return m

def d_nested_children(m: torch.nn.Module):
    m = decompose(m)
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
       # look for children from children... to the last child!
       for name, child in children.items():
            try:
                output[name] = d_nested_children(child)
            except TypeError:
                output[name] = d_nested_children(child)
    return output

In [151]:
def lr(m):
    for name, mod in m.named_children():
        if type(mod) == torch.nn.modules.conv.Conv2d:
            mod = tltorch.FactorizedConv.from_conv(
                mod, 
                rank=0.5, 
                decompose_weights=True, 
                factorization='tucker',
            )
            m._modules[name] = mod

In [164]:
def outer(layers):
    def lr(m):
        # i = 0
        for name, mod in m.named_children():
            if type(mod) == torch.nn.modules.conv.Conv2d:
                if name in layers:
                    mod = tltorch.FactorizedConv.from_conv(
                        mod, 
                        rank=0.5, 
                        decompose_weights=True, 
                        factorization='tucker',
                    )
                    m._modules[name] = mod
                # i += 1
    return lr

model = torch.load("/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best")
model.apply(outer(['conv1']))

ResNet(
  (conv1): FactorizedConv(
    in_channels=1, out_channels=64, kernel_size=(3, 3), rank=(4, 1, 1, 1), order=2, padding=[1, 1], bias=False
    (weight): TuckerTensor(shape=(64, 1, 3, 3), rank=(4, 1, 1, 1))
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
model = torch.load("/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best")
model.apply(lr)

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


ResNet(
  (conv1): FactorizedConv(
    in_channels=1, out_channels=64, kernel_size=(3, 3), rank=(4, 1, 1, 1), order=2, padding=[1, 1], bias=False
    (weight): TuckerTensor(shape=(64, 1, 3, 3), rank=(4, 1, 1, 1))
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )


In [7]:
def outer(layers):
    i = 0
    def lr(m):
        nonlocal i
        for l, (name, mod) in enumerate(m.named_children()):
            print(l)
            if type(mod) == torch.nn.modules.conv.Conv2d:
                if i in layers:
                    print('lr')
                    mod = tltorch.FactorizedConv.from_conv(
                        mod, 
                        rank=0.5, 
                        decompose_weights=True, 
                        factorization='tucker',
                    )
                    m._modules[name] = mod
                i+=1
    return lr

model = torch.load("/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best")
model.apply(outer([1]))

0
1
2
3
lr
4
0
1
2
3
4
0
1
0
0
1
2
3
4
0
1
2
3
4
0
1
0
0
1
2
3
4
0
1
2
3
4
0
1
0
0
1
2
3
4
0
1
2
3
4
0
1
0
1
2
3
4
5
6


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (shortcut): Sequential()
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): B

In [2]:
import torch
import tltorch

def decompose_conv(m):
    return tltorch.FactorizedConv.from_conv(
        m, 
        rank=0.5, 
        decompose_weights=True, 
        factorization='tucker',
    )

def low_rank_tucker(model, **kwargs):
    i = 0
    def nested_children(m: torch.nn.Module, layers=[1]):
        nonlocal i
        # print(i)
        children = dict(m.named_children())
        output = {}

        # this counts every layer
        # layer0 with child group0 with child conv0 will all have different numbers
        i+=1
        
        if children == {}:
            # this counts only children who don't have children themselfs
            # causing the same number to be used for nested modules 
            # layer0 with child group0 with child conv0 will all have the same number
            # i+=1

            # if module has no children; m is last child! :O
            return m
        else:
        # look for children from children... to the last child!
            for name, child in children.items():
                print(i, name, type(child))
                if type(child) == torch.nn.modules.conv.Conv2d and i in layers:
                    m._modules[name] = decompose_conv(child)
                try:
                    output[name] = nested_children(child, layers=layers)
                except TypeError:
                    output[name] = nested_children(child, layers=layers)
        return output
    out = nested_children(model, **kwargs)

path = "/home/jetzeschuurman/gitProjects/phd/tddl/artifacts/f_mnist/parn_18_d0.5_256_sgd_l0.1_g0.1/1629473591/cnn_best"
model = torch.load(path)
low_rank_tucker(model, layers=[6])
print(model)

1 conv1 <class 'torch.nn.modules.conv.Conv2d'>
2 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
3 layer1 <class 'torch.nn.modules.container.Sequential'>
4 0 <class 'tddl.models.resnet.PreActBlock'>
5 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
6 conv1 <class 'torch.nn.modules.conv.Conv2d'>
7 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
8 conv2 <class 'torch.nn.modules.conv.Conv2d'>
9 shortcut <class 'torch.nn.modules.container.Sequential'>
10 1 <class 'tddl.models.resnet.PreActBlock'>
11 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
12 conv1 <class 'torch.nn.modules.conv.Conv2d'>
13 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
14 conv2 <class 'torch.nn.modules.conv.Conv2d'>
15 shortcut <class 'torch.nn.modules.container.Sequential'>
16 layer2 <class 'torch.nn.modules.container.Sequential'>
17 0 <class 'tddl.models.resnet.PreActBlock'>
18 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
19 conv1 <class 'torch.nn.modules.conv.Conv2d'>
20 bn2 <cla

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


In [22]:
def func(inp, *args, default="value", **kwargs):
    print(inp)
    print(*args)
    print(default)
    print(**kwargs)

def func2(inp, *args, default="value", **kwargs):
    print("2", inp)
    print("2", *args)
    print("2", default)
    print("2", **kwargs)

    func(inp, *args, **kwargs)


In [23]:
func("a", "b", default="v")

a
b
v



In [24]:
func2("a", "b", default="v")

2 a
2 b
2 v
2
a
b
value



In [18]:
from tddl.factorizations import number_layers

out = number_layers(model, verbose=True, exclude=['layer1'])

0 conv1 <class 'torch.nn.modules.conv.Conv2d'>
1 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
2 layer1 <class 'torch.nn.modules.container.Sequential'>
2 layer2 <class 'torch.nn.modules.container.Sequential'>
3 0 <class 'tddl.models.resnet.PreActBlock'>
4 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
5 conv1 <class 'torch.nn.modules.conv.Conv2d'>
6 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
7 conv2 <class 'torch.nn.modules.conv.Conv2d'>
8 shortcut <class 'torch.nn.modules.container.Sequential'>
9 0 <class 'torch.nn.modules.conv.Conv2d'>
10 1 <class 'tddl.models.resnet.PreActBlock'>
11 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
12 conv1 <class 'torch.nn.modules.conv.Conv2d'>
13 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
14 conv2 <class 'torch.nn.modules.conv.Conv2d'>
15 shortcut <class 'torch.nn.modules.container.Sequential'>
16 layer3 <class 'torch.nn.modules.container.Sequential'>
17 0 <class 'tddl.models.resnet.PreActBlock'>
18 bn1 <class '

In [15]:
out

{'conv1': (1,
  Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
 'bn1': (2,
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
 'layer1': (3,
  {'0': (4,
    {'bn1': (5,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv1': (6,
      {'weight': (7, {'factors': (8, FactorList(
              (factor_0): Parameter containing: [torch.cuda.FloatTensor of size 64x49 (GPU 0)]
              (factor_1): Parameter containing: [torch.cuda.FloatTensor of size 64x49 (GPU 0)]
              (factor_2): Parameter containing: [torch.cuda.FloatTensor of size 3x2 (GPU 0)]
              (factor_3): Parameter containing: [torch.cuda.FloatTensor of size 3x2 (GPU 0)]
          ))})}),
     'bn2': (9,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv2': (10,
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'sh

In [13]:
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(49, 49, 2, 2), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(49, 49, 2, 2))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): B